<h1 align="center"> Aplicações em Processamento de Linguagem Natural </h1>
<h2 align="center"> Aula 09 - Geração Automática de Texto </h2>
<h3 align="center"> Prof. Fernando Vieira da Silva MSc.</h3>

<h2> 1. Introdução</h2>

<p>Enquanto na Extração de Informação a tarefa principal estava relacionada em compreender a escrita e tratar ambiguidades ou similaridades no texto, na geração automática de texto o desafio está em decidir como expressar uma dada informação em palavras.
    </p>
    
  <p>Há diversas aplicações práticas para NLG (do inglês, Natural Language Generation), como:</p>
  
  * Sumarização de documentos;
  * Geração de notícias;
  * Geração de texto para chatbots.
  
  <p>Nesta aula vamos abordar algumas técnicas para geração automática de texto, usando redes neurais recorrentes.</p>

<h2>2. Técnicas para Geração de Texto</h2>

<p> O objetivo da técnica de geração automática de texto é, dada uma sequência de n-gramas (ou caracteres, ou sentenças), prever qual será o próximo n-grama (ou caractere, ou sentença).
    </p>

<h2>3. Redes Neurais Recorrentes</h2>

<p>Uma rede neural recorrente é uma rede com um "loop", ou seja, uma rede que aproveita o aprendizado de uma amostra anterior para amostras seguintes. A figura abaixo, retirada de http://colah.github.io/posts/2015-08-Understanding-LSTMs/ ilustra sua arquitetura.
    </p>
    
![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)    

<p>Perceba que essa arquitetura de redes neurais é especialmente aplicável para problemas com sequências, uma vez que as amostras anteriores influenciam na aprendizagem.</p>

<p>Porém, as redes recorrentes convencionais tem limitações para utilizar informações aprendidas em etapas mais distantes (que acabam sendo atualizadas durante o treino). As redes neurais LSTM vieram para tentar sanar esse problema, como veremos à seguir.</p>

<h3>Redes LSTM</h3>

<p>As redes LSTM (Long-Short Term Memory, ou de Memória de curto e longo prazo, numa tradução livre), ao contrário das redes neurais recorrentes convencionais, que apenas atualizam o estado aprendido em etapas anteriores, trazem o conceito de memória, que é controlada pelo modelo.
    </p>
    <p>Desta forma, esse modelo não apenas armazena os estados aprendidos em etapas anteriores, mas decide o que deve ser aproveitado, descartado ou atualizado. Vejamos a ilustração abaixo, também extraída de http://colah.github.io/posts/2015-08-Understanding-LSTMs/.
    </p>
    
![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)    

<p>As etapas ilustradas são: </p>
1. *Qual estado de memória deve ser lembrado*: A primeira camada usa uma função sigmoid que retorna um valor entre 0 e 1 para decidir se o estado de memória vindo da iteração anterior deve ser mantido (1 para completamente mantido) ou esquecido (0 para completamente esquecido);
2. *Qual estado de memória deve ser atualizado*: Primeiramente usa-se uma função sigmoid para decidir quais valores serão atualizados com os novos dados da amostra atual, depois usa-se uma função tanh (que retorna um valor entre -1 e 1) para criar um novo vetor de estados da memória(denominado Ct) que poderá ser utilizado na iteração seguinte;
3. *Atualizar as células de memória antigas*: Combina os estados que devem ser lembrados, os estados que devem ser atualizados e os novos estados para atualizar as células de memória, que serão utilizadas na iteração seguinte;
4. *Decide a saída*: Por fim, a saída da iteração é feita através da combinação de uma função sigmoid e uma função tanh, considerando o estado atual da memória.

    

<h2>4. Usando LSTM para geração de texto</h2>

<p> Vamos nos basear no exemplo disponível em https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms, e vamos usar um LSTM para geração de texto.
    </p>

In [1]:
import keras
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.layers.core import Activation, Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers.core import Dense, Activation
import keras.utils as kutils

Using TensorFlow backend.


In [2]:
#import nltk
#nltk.corpus.gutenberg.fileids()


In [3]:
#raw_txt = nltk.corpus.gutenberg.raw("shakespeare-hamlet.txt") + \
#nltk.corpus.gutenberg.raw("shakespeare-caesar.txt") + \
#nltk.corpus.gutenberg.raw("shakespeare-macbeth.txt")

# Código adaptado de https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms
import pandas as pd
import os

curr_dir = '../input/'
all_headlines = []
for filename in os.listdir(curr_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(curr_dir + filename)
        all_headlines.extend(list(article_df.headline.values))
        break

all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

1214

In [4]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import string

all_sents = [[w.lower() for w in word_tokenize(sen) if not w in string.punctuation] \
             for sen in all_headlines]

x = []
y = []

print(all_sents[:10])

for sen in all_sents:
    for i in range(1, len(sen)):
        x.append(sen[:i])
        y.append(sen[i])
        

print(x[:10])
print(y[:10])



[['former', 'n.f.l', 'cheerleaders', '’', 'settlement', 'offer', '1', 'and', 'a', 'meeting', 'with', 'goodell'], ['e.p.a', 'to', 'unveil', 'a', 'new', 'rule', 'its', 'effect', 'less', 'science', 'in', 'policymaking'], ['the', 'new', 'noma', 'explained'], ['how', 'a', 'bag', 'of', 'texas', 'dirt', 'became', 'a', 'times', 'tradition'], ['is', 'school', 'a', 'place', 'for', 'self-expression'], ['commuter', 'reprogramming'], ['ford', 'changed', 'leaders', 'looking', 'for', 'a', 'lift', 'it', '’', 's', 'still', 'looking'], ['romney', 'failed', 'to', 'win', 'at', 'utah', 'convention', 'but', 'few', 'believe', 'he', '’', 's', 'doomed'], ['chain', 'reaction'], ['he', 'forced', 'the', 'vatican', 'to', 'investigate', 'sex', 'abuse', 'now', 'he', '’', 's', 'meeting', 'with', 'pope', 'francis']]
[['former'], ['former', 'n.f.l'], ['former', 'n.f.l', 'cheerleaders'], ['former', 'n.f.l', 'cheerleaders', '’'], ['former', 'n.f.l', 'cheerleaders', '’', 'settlement'], ['former', 'n.f.l', 'cheerleaders', 

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np

all_text = [c for sen in x for c in sen]
all_text += [c for c in y]

all_text.append('UNK') # Palavra desconhecida

words = list(set(all_text))
        
word_indexes = {word: index for index, word in enumerate(words)}      

max_features = len(word_indexes)

x = [[word_indexes[c] for c in sen] for sen in x]
y = [word_indexes[c] for c in y]

print(x[:10])
print(y[:10])

y = kutils.to_categorical(y, num_classes=max_features)

maxlen = max([len(sen) for sen in x])

print(maxlen)


[[367], [367, 927], [367, 927, 850], [367, 927, 850, 1340], [367, 927, 850, 1340, 2628], [367, 927, 850, 1340, 2628, 3235], [367, 927, 850, 1340, 2628, 3235, 1991], [367, 927, 850, 1340, 2628, 3235, 1991, 1473], [367, 927, 850, 1340, 2628, 3235, 1991, 1473, 357], [367, 927, 850, 1340, 2628, 3235, 1991, 1473, 357, 3110]]
[927, 850, 1340, 2628, 3235, 1991, 1473, 357, 3110, 2281]
23


In [6]:
x = pad_sequences(x, maxlen=maxlen)
x = pad_sequences(x, maxlen=maxlen)

print(x[:10,-10:])
print(y[:10,-10:])

[[   0    0    0    0    0    0    0    0    0  367]
 [   0    0    0    0    0    0    0    0  367  927]
 [   0    0    0    0    0    0    0  367  927  850]
 [   0    0    0    0    0    0  367  927  850 1340]
 [   0    0    0    0    0  367  927  850 1340 2628]
 [   0    0    0    0  367  927  850 1340 2628 3235]
 [   0    0    0  367  927  850 1340 2628 3235 1991]
 [   0    0  367  927  850 1340 2628 3235 1991 1473]
 [   0  367  927  850 1340 2628 3235 1991 1473  357]
 [ 367  927  850 1340 2628 3235 1991 1473  357 3110]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [7]:
print(x[:10,-10:])

for y_ in y:
    for i in range(len(y_)):
        if y_[i] != 0:
            print(i)

[[   0    0    0    0    0    0    0    0    0  367]
 [   0    0    0    0    0    0    0    0  367  927]
 [   0    0    0    0    0    0    0  367  927  850]
 [   0    0    0    0    0    0  367  927  850 1340]
 [   0    0    0    0    0  367  927  850 1340 2628]
 [   0    0    0    0  367  927  850 1340 2628 3235]
 [   0    0    0  367  927  850 1340 2628 3235 1991]
 [   0    0  367  927  850 1340 2628 3235 1991 1473]
 [   0  367  927  850 1340 2628 3235 1991 1473  357]
 [ 367  927  850 1340 2628 3235 1991 1473  357 3110]]
927
850
1340
2628
3235
1991
1473
357
3110
2281
3351
1683
854
357
689
839
3054
1763
37
658
2398
1881
689
3353
3231
357
683
2845
1312
3283
2751
357
2543
1436
1328
357
1114
243
857
2777
1778
536
2662
243
357
690
1205
1340
1780
1146
2662
2644
1683
1589
298
184
1866
1356
1231
48
513
1340
1780
1389
2315
3267
2546
2707
1683
1232
125
1564
2137
513
1340
1780
3110
2281
2465
2750
646
3312
1542
357
135
1812
1150
1824
2069
128
1643
729
691
2635
1658
1815
1613
922
293
562
3399
3

<p>Agora vamos criar nosso modelo de LSTM usando o Keras.</p>

In [8]:
embedding_size = 10

model = Sequential()
    
# Add Input Embedding Layer
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
    
# Add Hidden Layer 1 - LSTM Layer
model.add(LSTM(100))
model.add(Dropout(0.1))
    
# Add Output Layer
model.add(Dense(max_features, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 23, 10)            34420     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3442)              347642    
Total params: 426,462
Trainable params: 426,462
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(x, y, epochs=20, verbose=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20


In [11]:
import pickle

print("Saving model...")
model.save('shak-nlg.h5')

with open('shak-nlg-dict.pkl', 'wb') as handle:
    pickle.dump(word_indexes, handle)

with open('shak-nlg-maxlen.pkl', 'wb') as handle:
    pickle.dump(maxlen, handle)
print("Model Saved!")

Saving model...
Model Saved!


In [12]:
import pickle

model = keras.models.load_model('shak-nlg.h5')
maxlen = pickle.load(open('shak-nlg-maxlen.pkl', 'rb'))
word_indexes = pickle.load(open('shak-nlg-dict.pkl', 'rb'))

<p>Agora vejamos como o algoritmo prevê uma palavra simples.</p>

In [13]:
sample_seed = input()
sample_seed_vect = np.array([[word_indexes[c] if c in word_indexes.keys() else word_indexes['UNK'] \
                    for c in word_tokenize(sample_seed)]])

print(sample_seed_vect)

sample_seed_vect = pad_sequences(sample_seed_vect, maxlen=maxlen)

print(sample_seed_vect)

predicted = model.predict_classes(sample_seed_vect, verbose=0)

print(predicted)

def get_word_by_index(index, word_indexes):
    for w, i in word_indexes.items():
        if index == i:
            return w
        
    return None


for p in predicted:    
    print(get_word_by_index(p, word_indexes))

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

<p><b>Exercício 9</b>: Escreva um algoritmo que gera palavras até 100 palavras, com base em um texto curto de entrada (no máximo 5 palavras).</p>

In [14]:
sample_seed = input()
sample_seed_vect = [word_indexes[c] if c in word_indexes.keys() else word_indexes['UNK'] \
                    for c in word_tokenize(sample_seed)]

print(sample_seed_vect)

predicted=[]



while (len(sample_seed_vect)<100):
    predicted = model.predict_classes(pad_sequences([sample_seed_vect],maxlen=maxlen), verbose=0)
    sample_seed_vect.extend(predicted)
    

res = []
for w in sample_seed_vect:
    res.append(get_word_by_index(w,word_indexes))
    
print(' '.join(res))

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.